Inspired by Andrew Ng lectures

### Import Libraries

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm


### Load data

In [7]:
df = pd.read_csv('csv/ratings_reduced.csv')
print(f"In this dataset there are {len(df.gameid.unique())} games, {len(df.user.unique())} users and {len(df)} total votes")
df.head(2)

In this dataset there are 554 games, 148308 users and 4946868 total votes


,gameid,user,rating
0,188,-Johnny-,5.0
1,188,-Morphling-,7.0


In [8]:
# id of the game - title of the game
dfnames = pd.read_csv('csv/1000games_complete.csv')[['gameid', 'title']]
dfnames.head(2)

,gameid,title
0,224517,Brass: Birmingham
1,161936,Pandemic Legacy: Season 1


## Collaborative Filtering

### Preparation
First, Let's create a matrix with a game for each row and a user for each column

In [9]:
def transform(df):
    """ 
    Convert a 'flat' table to a matrix:
    rows: games (specifically gameids)
    columns: users
    Y matrix (#of games X #of users) stores the ratings
    R matrix (#of games X #of users) : 1/0. 1 if a user rated a game, 0 otherwise
    Ymean - average rating for every game
    Ynorm = Y - Ymean  # 
    
    """
    pivot_df = df.pivot(index='gameid', columns='user', values='rating')
    Y = pivot_df.values
    R = np.where(np.isnan(Y), 0, 1)  # 1 if a user rated a game, 0 otherwise
    Y = np.nan_to_num(Y, nan=0)  # convert to numpy array, nan values change for 0s
    
    # mean normalization
    mask = Y != 0
    row_means = np.ma.masked_array(Y, ~mask).mean(axis=1)
    Ymean = np.array([row_means]).T
    Ynorm = np.where(Y != 0, Y - Ymean, Y)  # only if a user rated a game, otherwise it remains 0
    return Y, Ymean, Ynorm, R

In [10]:
# Creating matricies
Y, Ymean, Ynorm, R = transform(df)

In [11]:
# average rating for the first 5 games in our matrix (not the best games)
Ymean[:5]

array([[7.37714084],
       [8.14358174],
       [7.13519132],
       [8.01598796],
       [8.04946373]])

### Cost Function


In [12]:
def cost_func(X, W, b, Y, R, lambda_):
    """
      Returns the cost
    
      X       : matrix of game features
      W       : matrix of user parameters
      b       : vector of user parameters
      Y       : matrix of user ratings of games
      R       : 1/0 matrix, 1 if the game rated by user, 0 otherwise
      lambda_ : regularization parameter

    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

### Initializing the parameters 
W, X, b

In [13]:
num_games, num_users = Y.shape  # number of games, number of users 
print(f"# of games: {num_games}, # of users: {num_users}")
num_features = 100

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(2604) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_games,  num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.1)

# of games: 554, # of users: 148308


### Training the Model
Let's train *collaborative filtering model* to learn **X, W, b**,  using TensorFlow

In [17]:
# Attention! My Laptop got hot...
iterations = 1000
lambda_ = 1
for iter in tqdm(range(iterations), total=iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cost_func(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # to see the process 
    if iter % 10 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:,.0f}")

  0%|          | 1/1000 [00:16<4:29:52, 16.21s/it]

Training loss at iteration 0: 6,567,081


  1%|          | 11/1000 [03:32<5:09:55, 18.80s/it]

Training loss at iteration 10: 5,144,078


  2%|▏         | 21/1000 [06:14<4:29:50, 16.54s/it]

...

 98%|█████████▊| 981/1000 [4:49:30<05:16, 16.65s/it]

Training loss at iteration 980: 186,586


 99%|█████████▉| 991/1000 [4:52:28<02:42, 18.09s/it]

Training loss at iteration 990: 186,428


100%|██████████| 1000/1000 [4:55:14<00:00, 17.71s/it]


### Make predictions
Now after learning **X, W, b**, we are able to make predictions

In [18]:
# Make a prediction using trained X,W and b
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()
pm = p + Ymean  # prediction matrix
pm.shape

(554, 148308)

#### Sandbox
it's just for me to practice - predictions for one particular user, and list of his rated games

In [19]:
def make_recommendations(prediction_matrix, df, user, number_of_recommendations=9):
    """ Recommendations for a given user """
    pivot_df = df.pivot(index='gameid', columns='user', values='rating')
    
    reconstructed_df = pd.DataFrame(prediction_matrix, index=pivot_df.index, columns=pivot_df.columns)
    reconstructed_R = pd.DataFrame(R, index=pivot_df.index, columns=pivot_df.columns)
    R # 1 or 0 (rated or not)
    user_df = pd.concat([reconstructed_df[user], reconstructed_R[user]], axis=1)
    user_df.columns = ['rating', 'rated']
    user_df_notrated = user_df.query('rated == 0').rating.sort_values(ascending=False)[:number_of_recommendations]
    
    # rated games
    user_df_rated = user_df.query('rated == 1').rating.sort_values(ascending=False)[:]
    
    # combine with names    
    user_df_notrated = pd.merge(user_df_notrated.to_frame(), dfnames, left_index=True, right_on='gameid', how='left')
    user_df_notrated = user_df_notrated[['rating', 'gameid', 'title']]
    
    # add rated games
    user_df_rated = pd.merge(user_df_rated.to_frame(), dfnames, left_index=True, right_on='gameid', how='left')
    user_df_rated = user_df_rated[['rating', 'gameid', 'title']]
    
    print("Rated games:")
    print(user_df_rated.title)
    print("And now recommendations:")
    recommendation = {'game':  None,
                     'rating': None,
                     }
    return user_df_notrated

In [20]:
# rated games by user 'axci' and recommendations below
axci = make_recommendations(pm, df, 'axci')
axci

Rated games:
2                                         Gloomhaven
90                                        Frosthaven
6                       Gloomhaven: Jaws of the Lion
0                                  Brass: Birmingham
193                                               Go
12     Through the Ages: A New Story of Civilization
10                                     Spirit Island
1                          Pandemic Legacy: Season 1
33                                           Barrage
30                                          Everdell
32                     Viticulture Essential Edition
14                               Great Western Trail
5                                  Terraforming Mars
7                                     Dune: Imperium
28                               Lost Ruins of Arnak
119                                        Codenames
69                                              Azul
128                                    Cartographers
97                               

,rating,gameid,title
631,10.115838,242705,Aeon Trespass: Odyssey
272,9.497493,251661,Oathsworn: Into the Deepwood
49,9.216685,314040,Pandemic Legacy: Season 0
88,9.203654,180263,The 7th Continent
389,9.201968,240980,Blood on the Clocktower
407,9.125516,170771,Sword & Sorcery
36,9.113900,175914,Food Chain Magnate
136,9.112481,271055,Dwellings of Eldervale
456,9.095228,169427,Middara: Unintentional Malum – Act 1


#### Function

In [21]:
# function that will create a flat dataframe with user - recommended gameid
def create_recommended_games(prediction_matrix, df, number_of_recommendations=10):
    """ 
    prediction_matrix - our trained matrix
    df -  dataframe from csv file ratings_reduced.csv (gameid-user-rating)
    """
    
    pivot_df = df.pivot(index='gameid', columns='user', values='rating')  # make a pivot table from the flat table
    R = np.where(np.isnan(pivot_df.values), 0, 1)  # 1 if a user rated a game, 0 otherwise
    
    reconstructed_df = pd.DataFrame(prediction_matrix, index=pivot_df.index, columns=pivot_df.columns)  # convert matrix to df 
    reconstructed_R = pd.DataFrame(R, index=pivot_df.index, columns=pivot_df.columns)  # convert R to df
    
    predictions = pd.DataFrame(columns=['prediction_rating', 'rated_bool', 'user'], index=pd.Index([], name='gameid'))  # empty df
    counter = 0
    for user in reconstructed_df.columns:
        user_df = pd.concat([reconstructed_df[user], reconstructed_R[user]], axis=1)
        user_df['user'] = user
        user_df.columns = ['prediction_rating', 'rated_bool', 'user']
        user_df_notrated = user_df.query('rated_bool == 0').sort_values(by='prediction_rating',ascending=False)[:number_of_recommendations]
        predictions = pd.concat([predictions, user_df_notrated])
        counter += 1
        if counter % 10000 == 0:
            print(f"{counter} out of {len(reconstructed_df.columns)} added")
   
    predictions['user'].to_csv('csv/rec_games.csv')  # save to csv
    return predictions

In [22]:
# Run the function to make predictions for every user
predictions = create_recommended_games(pm, df)


10000 out of 148308 added
20000 out of 148308 added
30000 out of 148308 added
40000 out of 148308 added
50000 out of 148308 added
60000 out of 148308 added
70000 out of 148308 added
80000 out of 148308 added
90000 out of 148308 added
100000 out of 148308 added
110000 out of 148308 added
120000 out of 148308 added
130000 out of 148308 added
140000 out of 148308 added


In [23]:
predictions['user'].to_csv('csv/rec_games.csv')

In [5]:
pr_user = pd.read_csv('csv/rec_games.csv')
print(len(pr_user))
pr_user.head(2)
#pr_user[:1000].to_csv('csv/rec_games_reduced.csv', index=False)  # just for testing

1483072


,gameid,user
0,295770,-=Yod@=-
1,251661,-=Yod@=-


In [93]:
# the final output 'rec_games.csv' I will use to load to User model of my Django framework